In [ ]:
import pandas as pd
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name='inference_url'
)


sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [4]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()



In [3]:
df = pd.read_csv(
's3://amazon-sagemaker-673945926093-us-west-2-d4ad45f48565/dzd-clo26by1ufxlcg/d4b94v6rej70y8/result/test/part-00000-bfa95f6f-3863-4cf9-9f75-49f1892d182f-c000.csv', on_bad_lines='skip' )


In [5]:
df = df.dropna(subset=['contents_final', 'label'])


In [6]:
df['contents_final'] = df['contents_final'].apply(lambda x : x[:500])

In [7]:
df['contents_length'] = df['contents_final'].apply(lambda x : len(x))

In [ ]:
prediction= [] 
for inputs in df['contents_final'].tolist():
    response = predictor.predict({"inputs": inputs})
    prediction.append(response)

In [ ]:
prediction_f = [] 
for ii in prediction:
    prediction_f.append(ii[0]["label"])

In [ ]:
df['prediction'] = prediction_f

In [ ]:
result =df[["contents_final", "label", "prediction" ]]

In [ ]:
result["convert"] =result["prediction"].apply(lambda x : int(x[-1]))

In [ ]:
result

In [14]:
from sklearn.metrics import precision_score, classification_report, confusion_matrix
import numpy as np

In [15]:
precision = precision_score(result["label"], result["convert"])


In [16]:
precision

0.9408284023668639